We'll make a video of the Johnson-Mehl process itself: point arriving and discs expanding.

Just like with the Pareto video, we need to colour cells by the "all-time adjacency" graph.

In [152]:
import numpy as np
from scipy.spatial import KDTree
from tqdm import tqdm, trange
import networkx
import colorspace
from PIL import Image, ImageColor, ImageDraw
import sys
import json
import os

from unconstrained import sample_points, prune_arrivals
from draw_jm import get_adjacency, colour_graph, get_ball_pixels, assign_cells
from draw_jm import assign_cells as assign_with_progress_bar

In [153]:
def get_arrival_times( rho, max_time=1.0, R=0 ):
    # PROBLEM (major-ish):
    # The "start again if there are more than Nmax arrivals" method
    # means our arrival times don't exactly have the distribution of homogeneous
    # Poisson arrivals. Instead I suppose I should generate some new samples.
    rate = rho*(1+2*R)**2
    Nmax = int(max_time*rate + 2*np.sqrt(max_time*rate)) # Two standard deviations above the mean
    interarrival_times = np.random.exponential(scale=1/rate,size=Nmax)
    arrival_times = np.cumsum(interarrival_times)
    too_late = np.searchsorted(arrival_times,max_time,side='right') # First index where the arrival time is at least max_time
    while too_late == Nmax: # This will be the case if we are unlucky and Nmax points arrived before time max_time. We'll just generate more points.
        interarrival_times = np.append(interarrival_times, np.random.exponential(scale=1/rate,size=Nmax))
        arrival_times = np.cumsum(interarrival_times)
        too_late = np.searchsorted(arrival_times,max_time,side='right') # First index where the arrival time is at least max_time
    return arrival_times[:too_late].copy()
    # N = np.random.poisson(lam=rho*max_time*(1+2*R)**2)
    # return np.sort(np.random.uniform(low=0.0, high=max_time, size=N))
# # Needed redefining because of my foolishly using a global variable (rng)
# # to define the version of this function in unconstrained.py.

# We also need a new assign_cells so it doesn't have a progress bar.
def assign_cells( seeds, times, img_size, T=1.0 ):
    """
    Assigns all the pixels in an img_size x img_size picture
    to their respective Johnson-Mehl cells.
    T should be a decent upper bound on the coverage time - smaller T
    means we check fewer points.
    This is a modified version of Moulinec's algorithm,
    in which we assign things which were covered by time T,
    and leave the rest unassigned.
    """
    min_cov_times = np.full((img_size,img_size),np.inf) # running minimum coverage times
    assignments = np.full((img_size,img_size),-1,dtype=int) # everything uncovered is assigned to a separate class.

    for i in range(len(times)):
        xi = seeds[i]
        ti = times[i]
        indices, d2s = get_ball_pixels(xi, T-ti, img_size)
        for k, ij_pair in enumerate(indices):
            cov_time = np.sqrt(d2s[k])/img_size + ti
            if cov_time < min_cov_times[ij_pair]:
                assignments[ij_pair] = i
                min_cov_times[ij_pair] = cov_time
    return assignments

def assign_cells_v2(seeds, times, img_size, T=1.0):
    """
    Assigns all the pixels in an img_size x img_size picture
    to their respective Johnson-Mehl cells.
    T should be a decent upper bound on the coverage time - smaller T
    means we check fewer points.
    This is a simplified version of Moulinec's algorithm,
    in which we set T (his t_0) greater than the coverage time.
    His choice of t_0 and extra step probably does improve the speed,
    but I prefer the simplicity of this method, especially as we have
    a decent estimate on the coverage time when rho is large.
    """
    min_cov_times = np.full((img_size,img_size),np.inf) # running minimum coverage times
    assignments = np.empty((img_size,img_size),dtype=int)
    for i in trange(len(times),leave=False):
        xi = seeds[i]
        ti = times[i]
        indices, d2s = get_ball_pixels(xi, T-ti, img_size)
        for k, ij_pair in enumerate(indices):
            cov_time = np.sqrt(d2s[k])/img_size + ti
            if cov_time < min_cov_times[ij_pair]:
                assignments[ij_pair] = i
                min_cov_times[ij_pair] = cov_time
    return assignments, min_cov_times

Now that I have a fast method, it might even be possible to make an interactive slider controlling the time.

### "merging" method

In [154]:
def merge_jm_arrivals(t1,l1,t2,l2):
    """
    Given two sets of arrival times and locations from a time-homogeneous PPP,
    merges them into a single pair.

    The arguments are all numpy arrays, and both t1 and t2 should be sort
    """
    totallen = len(t1)+len(t2)
    outtimes = np.empty(totallen)
    outseeds = np.empty((totallen,2))
    i1 = 0
    i2 = 0
    while i1 < len(t1) and i2 < len(t2):
        if t1[i1] < t2[i2]:
            outtimes[i1+i2] = t1[i1]
            outseeds[i1+i2] = l1[i1]
            i1 += 1
        else:
            outtimes[i1+i2] = t2[i2]
            outseeds[i1+i2] = l2[i2]
            i2 += 1
    if i1 == len(t1):
        outtimes[i1+i2:] = t2[i2:]
        outseeds[i1+i2:] = l2[i2:]
    else:
        outtimes[i1+i2:] = t1[i1:]
        outseeds[i1+i2:] = l1[i1:]
    return outtimes, outseeds

In [155]:
def batched_arrivals(batch_rho, batches):
    times_processes = []
    seeds_processes = []
    rho = batch_rho * batches
    for i in trange(batches,leave=False):
        times = get_arrival_times(batch_rho)
        seeds = sample_points(len(times))
        arrived = prune_arrivals(times, seeds) # pruning may be slow if rho is very large.
        times_processes.append(times[arrived])
        seeds_processes.append(seeds[arrived])
    times = np.concatenate(times_processes)
    seeds = np.concatenate(seeds_processes)
    indices = np.argsort(times)
    times = times[indices]
    seeds = seeds[indices]
    print(f'We have {len(times)} arrivals in the merged processes. Pruning now.')
    arrived = prune_arrivals(times, seeds) # pruning may be slow if rho is very large.
    times = times[arrived]
    seeds = seeds[arrived]
    print(f'We have a total of {len(times)} arrivals for an arrival rate of {rho:.2e}.')
    return times, seeds

In [156]:
# for i in range(8,100):
#     print(f'Generating sample {i}')
#     times, seeds = batched_arrivals(1e7,10)
#     np.savez(f'arrivals/rho1e8-jm-{i}',times=times,seeds=seeds)

In [157]:
# times_processes = []
# seeds_processes = []
# for i in range(8):
#     times_processes.append(np.load(f'arrivals/rho1e8-jm-times{i}.npy'))
#     seeds_processes.append(np.load(f'arrivals/rho1e8-jm-seeds{i}.npy'))
# for i in range(8,16):
#     both = np.load(f'arrivals/rho1e8-jm-{i}.npz')
#     times_processes.append(both['times'])
#     seeds_processes.append(both['seeds'])
# times = np.concatenate(times_processes)
# seeds = np.concatenate(seeds_processes)
# indices = np.argsort(times)
# times = times[indices]
# seeds = seeds[indices]
# print(f'We have {len(times)} arrivals in the merged processes. Pruning now.')
# arrived = prune_arrivals(times, seeds) # pruning may be slow if rho is very large.
# times = times[arrived]
# seeds = seeds[arrived]
# print(f'We have a total of {len(times)} arrivals for an arrival rate of {16*1e8}.')

## New method

This is about 50 times faster than the old method.

In [158]:
def produce_video_frames(rho,resolution,nframes,fileprefix,rngseed=None,circlepause=40):
    circlesize = int(resolution / 15)
    circlewidth = int(resolution/80)
    if rngseed:
        np.random.seed(rngseed)
    ratios = np.linspace(0,2.0,num=nframes,endpoint=False)

    times = get_arrival_times(rho)
    seeds = sample_points(len(times))
    arrived = prune_arrivals(times, seeds) # pruning may be slow if rho is very large.
    print(f'{len(arrived)} out of {len(times)} seeds germinated.')
    times = times[arrived]
    seeds = seeds[arrived]

    stronglaw = ( (2*np.log(rho) + 4*np.log(np.log(rho))) / (np.pi*rho) )**(1/3)

    print("First, we generate the final tessellation (which might take a moment)...")
    I, cov_times = assign_cells_v2(seeds, times, resolution, T = 1.5*stronglaw)
    print("Cells created, now we will colour them.")
    supG = get_adjacency(I)
    print(supG)
    colours = colour_graph(supG)
    print(f'We have a {max(colours.values())+1}-colouring of the cells.')

    print("Now we can start to generate frames.")
    c = colorspace.hcl_palettes().get_palette(name="Emrld")
    hex_colours = c(max(colours.values())+1)
    rgb_colours = [ImageColor.getcolor(col,"RGB") for col in hex_colours]
    
    final_frame = np.full((resolution, resolution, 3), 300, dtype=np.uint8)
    for x in range(resolution):
        for y in range(resolution):
            final_frame[x,y,:] = rgb_colours[colours[I[x,y]]]
    current_frame = np.copy(final_frame)
    
    prev_uncovered = np.full((resolution,resolution),False)
    
    progress = trange(len(ratios),leave=False)
    for i in progress:
        t = ratios[i]
        current_time = t*stronglaw
        progress.set_description(f'Working on t={t:.4f}')
        current_frame = np.copy(final_frame)
        uncovered = (cov_times > current_time)
        for channel in range(3):
            current_frame[:,:,channel][uncovered] = 255
        if not (True in uncovered): # This condition is met if all points are covered.
            print("We've found the first covered frame, so we're nearly done!")
            prev_uncovered = cov_times > (ratios[i-1]*stronglaw)
            # image still holds the previous frame
            draw = ImageDraw.Draw(image)
            for x in range(resolution):
                for y in range(resolution):
                    if prev_uncovered[x,y]:
                        if x >= 0.1*resolution and x <= 0.9*resolution and y >= 0.1*resolution and y <= 0.9*resolution:
                            print("!!!!!!!!!!!!!!!!!!!!!!!!!")
                            print("The last covered point was in the interior! This is pretty rare, you're lucky to see it!")
                            print("!!!!!!!!!!!!!!!!!!!!!!!!!")
                        draw.ellipse((y-circlesize,x-circlesize,y+circlesize,x+circlesize),outline=(255,0,0),width=circlewidth)
            for s in range(circlepause):
                image.save(fileprefix+str(ratios[i-1])+'-'+str(s)+'.png')
            image = Image.fromarray(final_frame)
            for j in range(i,min(len(ratios),i+circlepause)):
                image.save(fileprefix+str(ratios[j])+'.png') # the remaining frames all look the same.
            break
        else:
            # The frame is uncovered, so draw it and move on.
            image = Image.fromarray(current_frame)
            image.save(fileprefix+str(t)+'.png')
            prev_uncovered = np.copy(uncovered)
    
    print("Done! Go and find your frames in the frames/ folder")
    return

In [159]:
# for SEED in range(16,101):
#     print(f'Making video number {SEED}')
#     produce_video_frames(100000000,1080,600,f'frames/video{SEED}-',rngseed=SEED)
#     print(f'Finished making video number {SEED}\n')

## Step-by-step:

In [188]:
# fileprefix = "frames/jm-"
rho = 10000
# resolution = 1080
# nframes = 500
# circlepause = 200 # How many frames to pause for with the circle around the last covered point
circlesize = int(resolution / 15)
circlewidth = int(resolution/80)

# ratios = np.linspace(0,2.0,num=nframes,endpoint=False)

times = get_arrival_times(rho)
seeds = sample_points(len(times))
arrived = prune_arrivals(times, seeds) # pruning may be slow if rho is very large.
print(f'{len(arrived)} out of {len(times)} seeds germinated.')
times = times[arrived]
seeds = seeds[arrived]

stronglaw = ( (2*np.log(rho) + 4*np.log(np.log(rho))) / (np.pi*rho) )**(1/3)

382 out of 9863 seeds germinated.


In [189]:
fileprefix = "frames/jm-"
resolution = 1080
nframes = 1200
circlepause = 200 # How many frames to pause for with the circle around the last covered point
circlesize = int(resolution / 15)
circlewidth = int(resolution/80)

ratios = np.linspace(0,1.5,num=nframes,endpoint=False)

stronglaw = ( (2*np.log(rho) + 4*np.log(np.log(rho))) / (np.pi*rho) )**(1/3)

In [190]:
print("First, we generate the final tessellation (which might take a moment)...")
I, cov_times = assign_cells_v2(seeds, times, resolution, T = 2.0*stronglaw)
print("Cells created, now we will compute the adjacency graph.")
supG = get_adjacency(I)
print(supG) # This graph is planar, we should be able to 4-colour it. However, I don't think there's an efficient algorithm that guarantees a 4-colouring.

First, we generate the final tessellation (which might take a moment)...


Cells created, now we will compute the adjacency graph.
Graph with 382 nodes and 1056 edges


In [191]:
### Save the adjacency graph, go and colour it in another notebook with Sagemath, then load the colouring here.
networkx.write_adjlist(supG, 'supG.adjlist')

### WAIT - remember to run the other notebook to colour the graph

In [192]:
def keystoint(x):
    return {int(k):v for k, v in x} # Convert keys from str to int.

with open('colouring.json','r') as colfile:
    colours = json.load(colfile,object_pairs_hook=keystoint)

In [193]:
print("Now we can start to generate frames.")
# Delete the old frames:
os.system('rm frames/jm-*')
# c = colorspace.hcl_palettes().get_palette(name="Emrld") # Needs a new colour scheme.
# hex_colours = c(max(colours.values())+1)
# rgb_colours = [ImageColor.getcolor(col,"RGB") for col in hex_colours]
rgb_colours = [
    (55, 126, 184),
    (152, 78, 163),
    (255,127,0),
    (255,255,51),
    (153,153,153)
]

final_frame = np.full((resolution, resolution, 3), 300, dtype=np.uint8)
for x in range(resolution):
    for y in range(resolution):
        final_frame[x,y,:] = rgb_colours[colours[I[x,y]]]
current_frame = np.copy(final_frame)

prev_uncovered = np.full((resolution,resolution),False)

progress = trange(len(ratios),leave=False)
for i in progress:
    t = ratios[i]
    current_time = t*stronglaw
    progress.set_description(f'Working on t={t:.4f}')
    current_frame = np.copy(final_frame)
    uncovered = (cov_times > current_time)
    for channel in range(3):
        current_frame[:,:,channel][uncovered] = 255
    if not (True in uncovered): # This condition is met if all points are covered.
        print("We've found the first covered frame, so we're nearly done!")
        prev_uncovered = cov_times > (ratios[i-1]*stronglaw)
        # image still holds the previous frame
        draw = ImageDraw.Draw(image)
        for x in range(resolution):
            for y in range(resolution):
                if prev_uncovered[x,y]:
                    draw.ellipse((y-circlesize,x-circlesize,y+circlesize,x+circlesize),outline=(255,0,0),width=circlewidth)
        for s in range(circlepause):
            image.save(f'{fileprefix}{ratios[i-1]:.5f}'+'-'+str(s)+'.png')
        image = Image.fromarray(final_frame)
        for j in range(i,min(len(ratios),i+circlepause)):
            image.save(f'{fileprefix}{ratios[j]:.5f}.png') # the remaining frames all look the same.
        break
    else:
        # The frame is uncovered, so draw it and move on.
        image = Image.fromarray(current_frame)
        image.save(f'{fileprefix}{t:.5f}.png')
        prev_uncovered = np.copy(uncovered)

print("Done! Go and find your frames in the frames/ folder")

Now we can start to generate frames.


rm: cannot remove 'frames/*': No such file or directory
Working on t=1.2612:  84%|███████████████   | 1008/1200 [00:36<00:07, 27.28it/s]

We've found the first covered frame, so we're nearly done!


Done! Go and find your frames in the frames/ folder


Then go and stitch your video together with `ffmpeg -framerate 25 -pattern_type glob -i 'frames/jm-*.png' -c:v libx264 JM-VIDEO.mp4`

Note: the boundary effects are very strong. A quick estimate using (2.13) of the paper suggests that there is around a 2.5 percent chance that the last covered region will be in an area unaffected by the boundary.

Most of time time the last covered region seems to actually touch the boundary.